In [1]:
# Digital Economy Transformation Analysis - Project Setup
# Testing environment and imports

import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

print("All packages imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Requests version: {requests.__version__}")

All packages imported successfully!
Pandas version: 2.2.2
Requests version: 2.32.3


In [ ]:
All packages imported successfully!
Pandas version: 2.2.2
Requests version: 2.32.3

In [3]:
# Test World Bank API with simple call
import requests

def test_world_bank_api():
    url = "https://api.worldbank.org/v2/country/USA/indicator/NY.GDP.PCAP.CD"
    params = {'format': 'json', 'date': '2020:2023'}
    
    response = requests.get(url, params=params)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print(f"Data received: {len(data[1])} records")
        print("Sample record:", data[1][0])
        return True
    return False

# Test the API
test_world_bank_api()

Status Code: 200
Data received: 4 records
Sample record: {'indicator': {'id': 'NY.GDP.PCAP.CD', 'value': 'GDP per capita (current US$)'}, 'country': {'id': 'US', 'value': 'United States'}, 'countryiso3code': 'USA', 'date': '2023', 'value': 82769.4122114216, 'unit': '', 'obs_status': '', 'decimal': 1}


True

In [7]:
# Test BigQuery connection
import os
from google.cloud import bigquery

# Set the correct path to your service account key (go up one directory)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../digital-economy-analysis-6a0aae99b01f.json'

# Initialize BigQuery client
client = bigquery.Client(project='digital-economy-analysis')

print("BigQuery client initialized successfully!")
print(f"Project: {client.project}")

# Test query
query = "SELECT 1 as test_value"
result = client.query(query).result()
for row in result:
    print(f"Test query result: {row.test_value}")

BigQuery client initialized successfully!
Project: digital-economy-analysis
Test query result: 1


In [15]:
# Define indicators for digital economy analysis

indicators = {
    # Economic Development
    'gdp_per_capita': 'NY.GDP.PCAP.CD',
    'gdp_growth': 'NY.GDP.MKTP.KD.ZG',
    
    # Digital Infrastructure  
    'internet_users': 'IT.NET.USER.ZS',
    'mobile_subs': 'IT.CEL.SETS.P2',
    'broadband_subs': 'IT.NET.BBND.P2',
    
    # Financial Inclusion
    'bank_accounts': 'FX.OWN.TOTL.ZS',
    
    # Human Development
    'tertiary_education': 'SE.TER.ENRR',
    'urban_population': 'SP.URB.TOTL.IN.ZS'
}

print("Digital Economy Indicators:")
for name, code in indicators.items():
    print(f"  {name}: {code}")
    

Digital Economy Indicators:
  gdp_per_capita: NY.GDP.PCAP.CD
  gdp_growth: NY.GDP.MKTP.KD.ZG
  internet_users: IT.NET.USER.ZS
  mobile_subs: IT.CEL.SETS.P2
  broadband_subs: IT.NET.BBND.P2
  bank_accounts: FX.OWN.TOTL.ZS
  tertiary_education: SE.TER.ENRR
  urban_population: SP.URB.TOTL.IN.ZS


In [17]:
# World Bank Data Collection Function
import pandas as pd
import requests
import time

def fetch_world_bank_indicator(indicator_code, countries='all', start_year=2010, end_year=2023):
    """
    Fetch data for a specific World Bank indicator
    
    Args:
        indicator_code: World Bank indicator code (e.g., 'NY.GDP.PCAP.CD')
        countries: 'all' or specific country codes like 'USA;GBR;DEU'
        start_year: Starting year for data
        end_year: Ending year for data
    
    Returns:
        pandas DataFrame with cleaned data
    """
    
    url = f"https://api.worldbank.org/v2/country/{countries}/indicator/{indicator_code}"
    params = {
        'format': 'json',
        'date': f'{start_year}:{end_year}',
        'per_page': 1000  # Handle pagination
    }
    
    print(f"Fetching data for {indicator_code}...")
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        
        # Check if we have data
        if len(data) < 2 or not data[1]:
            print(f"  No data returned for {indicator_code}")
            return pd.DataFrame()
        
        # Convert to DataFrame
        records = data[1]
        df = pd.DataFrame(records)
        
        # Clean and structure the data
        if not df.empty:
            df = df[['country', 'date', 'value']].copy()
            df['country_name'] = df['country'].apply(lambda x: x['value'])
            df['country_code'] = df['country'].apply(lambda x: x['id'])
            df = df[['country_code', 'country_name', 'date', 'value']].copy()
            df['date'] = pd.to_numeric(df['date'])
            df['indicator'] = indicator_code
            
            # Remove rows with null values
            df = df.dropna(subset=['value'])
            
            print(f"  Retrieved {len(df)} records")
            return df
        
    except Exception as e:
        print(f"  Error fetching {indicator_code}: {str(e)}")
        return pd.DataFrame()

# Test the function with GDP per capita
test_df = fetch_world_bank_indicator('NY.GDP.PCAP.CD', countries='USA;GBR;DEU', start_year=2020, end_year=2023)
print("\nSample data:")
print(test_df.head())

Fetching data for NY.GDP.PCAP.CD...
  Retrieved 12 records

Sample data:
  country_code    country_name  date         value       indicator
0           DE         Germany  2023  54343.226508  NY.GDP.PCAP.CD
1           DE         Germany  2022  49686.115458  NY.GDP.PCAP.CD
2           DE         Germany  2021  52265.654162  NY.GDP.PCAP.CD
3           DE         Germany  2020  47379.765195  NY.GDP.PCAP.CD
4           GB  United Kingdom  2023  49463.855462  NY.GDP.PCAP.CD


In [19]:
# Collect all indicators
all_data = []

for indicator_name, indicator_code in indicators.items():
    print(f"\n--- Collecting {indicator_name} ---")
    
    # Fetch data for all countries
    df = fetch_world_bank_indicator(
        indicator_code=indicator_code,
        countries='all',
        start_year=2010,
        end_year=2023
    )
    
    if not df.empty:
        df['indicator_name'] = indicator_name
        all_data.append(df)
        print(f"✓ Success: {len(df)} records for {indicator_name}")
    else:
        print(f"✗ Failed: No data for {indicator_name}")
    
    # Wait a second between requests
    time.sleep(1)

print(f"\n=== Collection Complete ===")
print(f"Total indicators collected: {len(all_data)}")

# Combine all data into one DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"Total records: {len(combined_df)}")
    print(f"Countries: {combined_df['country_code'].nunique()}")
    print(f"Date range: {combined_df['date'].min()} - {combined_df['date'].max()}")
else:
    print("No data collected")


--- Collecting gdp_per_capita ---
Fetching data for NY.GDP.PCAP.CD...
  Retrieved 984 records
✓ Success: 984 records for gdp_per_capita

--- Collecting gdp_growth ---
Fetching data for NY.GDP.MKTP.KD.ZG...
  Retrieved 984 records
✓ Success: 984 records for gdp_growth

--- Collecting internet_users ---
Fetching data for IT.NET.USER.ZS...
  Retrieved 357 records
✓ Success: 357 records for internet_users

--- Collecting mobile_subs ---
Fetching data for IT.CEL.SETS.P2...
  Retrieved 915 records
✓ Success: 915 records for mobile_subs

--- Collecting broadband_subs ---
Fetching data for IT.NET.BBND.P2...
  Retrieved 929 records
✓ Success: 929 records for broadband_subs

--- Collecting bank_accounts ---
Fetching data for FX.OWN.TOTL.ZS...
  Retrieved 139 records
✓ Success: 139 records for bank_accounts

--- Collecting tertiary_education ---
Fetching data for SE.TER.ENRR...
  Retrieved 863 records
✓ Success: 863 records for tertiary_education

--- Collecting urban_population ---
Fetching dat

In [21]:
# Assess data 
print("=== DATA COLLECTION SUMMARY ===")
print(f"Total records: {len(combined_df):,}")
print(f"Unique countries: {combined_df['country_code'].nunique()}")
print(f"Date range: {combined_df['date'].min()} - {combined_df['date'].max()}")
print(f"Indicators collected: {combined_df['indicator_name'].nunique()}")

print("\n=== INDICATORS BREAKDOWN ===")
indicator_counts = combined_df['indicator_name'].value_counts()
print(indicator_counts)

print("\n=== DATA COMPLETENESS ===")
# Check data per indicator
completeness = combined_df.groupby('indicator_name').agg({
    'value': 'count',
    'country_code': 'nunique'
}).round(0)
completeness.columns = ['Total Records', 'Countries with Data']
print(completeness)

=== DATA COLLECTION SUMMARY ===
Total records: 6,157
Unique countries: 71
Date range: 2010 - 2023
Indicators collected: 8

=== INDICATORS BREAKDOWN ===
indicator_name
urban_population      986
gdp_per_capita        984
gdp_growth            984
broadband_subs        929
mobile_subs           915
tertiary_education    863
internet_users        357
bank_accounts         139
Name: count, dtype: int64

=== DATA COMPLETENESS ===
                    Total Records  Countries with Data
indicator_name                                        
bank_accounts                 139                   37
broadband_subs                929                   70
gdp_growth                    984                   71
gdp_per_capita                984                   71
internet_users                357                   27
mobile_subs                   915                   70
tertiary_education            863                   69
urban_population              986                   71


In [23]:
combined_df.head(10)

,country_code,country_name,date,value,indicator,indicator_name
0,ZH,Africa Eastern and Southern,2023,1659.515290,NY.GDP.PCAP.CD,gdp_per_capita
1,ZH,Africa Eastern and Southern,2022,1628.024526,NY.GDP.PCAP.CD,gdp_per_capita
2,ZH,Africa Eastern and Southern,2021,1522.590088,NY.GDP.PCAP.CD,gdp_per_capita
3,ZH,Africa Eastern and Southern,2020,1344.080962,NY.GDP.PCAP.CD,gdp_per_capita
4,ZH,Africa Eastern and Southern,2019,1493.780445,NY.GDP.PCAP.CD,gdp_per_capita
5,ZH,Africa Eastern and Southern,2018,1538.924188,NY.GDP.PCAP.CD,gdp_per_capita
6,ZH,Africa Eastern and Southern,2017,1520.171298,NY.GDP.PCAP.CD,gdp_per_capita
7,ZH,Africa Eastern and Southern,2016,1329.777824,NY.GDP.PCAP.CD,gdp_per_capita
8,ZH,Africa Eastern and Southern,2015,1479.564123,NY.GDP.PCAP.CD,gdp_per_capita
9,ZH,Africa Eastern and Southern,2014,1656.107642,NY.GDP.PCAP.CD,gdp_per_capita


## Data Quality Insights:
- Strong indicators: GDP, urban population, mobile/broadband (900+ records)
- Weak indicators: Bank accounts (139), internet users (357) - limited country coverage
- 71 countries with data (good sample size)

In [26]:
# Save raw data CSV in data folder
print("Saving data backup...")
combined_df.to_csv('../data/world_bank_raw_data.csv', index=False)
print("✓ Saved to data/world_bank_raw_data.csv")

# Also save indicators list
indicators_df = pd.DataFrame(list(indicators.items()), 
                           columns=['indicator_name', 'indicator_code'])
indicators_df.to_csv('../data/indicators_reference.csv', index=False)
print("✓ Saved indicators reference")

Saving data backup...
✓ Saved to data/world_bank_raw_data.csv
✓ Saved indicators reference


In [28]:
print("Uploading to BigQuery...")

# Configure upload
table_id = f"{client.project}.world_bank_data.raw_indicators"

# Upload data
job = combined_df.to_gbq(
    destination_table='world_bank_data.raw_indicators',
    project_id='digital-economy-analysis',
    if_exists='replace',
    progress_bar=True
)

print("✓ Data uploaded to BigQuery!")
print(f"Table: {table_id}")

Uploading to BigQuery...


/var/folders/0t/861tcrxd0m3dvztgzk5nkr3h0000gn/T/ipykernel_6576/2567285009.py:7: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  job = combined_df.to_gbq(
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 7810.62it/s]

✓ Data uploaded to BigQuery!
Table: digital-economy-analysis.world_bank_data.raw_indicators


In [32]:
# Import Library
import pandas_gbq

In [34]:
# Verify data in BigQuery
print("Verifying BigQuery upload...")

query = """
SELECT 
    indicator_name,
    COUNT(*) as record_count,
    COUNT(DISTINCT country_code) as country_count,
    MIN(date) as earliest_year,
    MAX(date) as latest_year
FROM `digital-economy-analysis.world_bank_data.raw_indicators`
GROUP BY indicator_name
ORDER BY record_count DESC
"""

result_df = pandas_gbq.read_gbq(query, project_id='digital-economy-analysis')
print("\nBigQuery Table Summary:")
print(result_df)

Verifying BigQuery upload...
Downloading: 100%|█████████████████████████████████████████████████████████████|

BigQuery Table Summary:
       indicator_name  record_count  country_count  earliest_year  latest_year
0    urban_population           986             71           2010         2023
1      gdp_per_capita           984             71           2010         2023
2          gdp_growth           984             71           2010         2023
3      broadband_subs           929             70           2010         2023
4         mobile_subs           915             70           2010         2023
5  tertiary_education           863             69           2010         2023
6      internet_users           357             27           2010         2023
7       bank_accounts           139             37           2011         2022


In [ ]:
# Record progress
print("Phase 2 Complete: Data Collection")
print("- ✅ World Bank API integration")
print("- ✅ 6,157 records collected") 
print("- ✅ Data uploaded to BigQuery")
print("- ✅ Data backup saved locally")